In [1]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=2395008e01ada296212d2ae958afc7b5cb80f337aeb177e62aa123a7b17aa3b1
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [2]:
from tensorflow.keras.layers import Input, Dense, GaussianNoise, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from google.colab import drive
from datetime import datetime
from tensorflow import keras
from pyswarm import pso
import pandas as pd
import numpy as np
import time

In [3]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [4]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [5]:
filename = 'RSV_data.csv'
RSV_data = pd.read_csv(path + filename)

set_index(RSV_data)

tm_data = RSV_data

RSV_data = RSV_data.drop(columns=['RSV'])

RSV_data['RSV'] = tm_data['RSV']

RSV_data

,gdp_growth,volume,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)",...,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,"('APARCH', 'studentst', 1)",RSV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,28984.929215,2.945453e+09,0.039601,0.038301,0.039601,0.038301,0.040044,0.039199,0.038388,0.038388,...,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.037128,0.032740
2000-02-03,28984.929215,2.935907e+09,0.037979,0.038355,0.037979,0.038355,0.041000,0.040608,0.038633,0.038633,...,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.038336,0.033467
2000-02-04,28984.929215,3.274647e+09,0.036791,0.037755,0.036791,0.037755,0.038214,0.040159,0.036990,0.036990,...,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.049778,0.032057
2000-02-07,28984.929215,3.413593e+09,0.046593,0.055072,0.046593,0.055072,0.047968,0.057512,0.046474,0.046474,...,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.065732,0.034853
2000-02-08,28984.929215,2.868803e+09,0.049131,0.041984,0.049131,0.041984,0.049560,0.042487,0.046711,0.046711,...,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.043373,0.035254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000000,1.609587e+08,0.000120,-0.000665,0.000120,-0.000665,0.000456,-0.000424,0.000121,0.000121,...,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,-0.004076,0.000006
2024-08-27,0.000000,9.767973e+07,-0.003889,-0.004798,-0.003889,-0.004798,-0.003539,-0.002953,-0.004368,-0.004368,...,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,-0.005604,-0.000017
2024-08-28,0.000000,2.498703e+08,0.000280,-0.001148,0.000280,-0.001148,0.000449,-0.000383,0.000483,0.000483,...,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,-0.001642,0.000048


# Split train, validation and test data

In [6]:
# def split_data(data, target, train_end =datetime(2020, 6, 30), test_start=datetime(2020, 10, 1), test_size=0.1):
#   test_data = data.loc[test_start:]

#   train_val = data.loc[:train_end]
#   train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

#   X_train = train_data.drop(columns=[target])
#   y_train = train_data[target]

#   X_val = val_data.drop(columns=[target])
#   y_val = val_data[target]

#   X_test = test_data.drop(columns=[target])
#   y_test = test_data[target]

#   return np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val), np.array(X_test), np.array(y_test)

In [7]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

# Rolling Mechanism

In [8]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

## RSV

In [9]:
# RSV_X_train, RSV_y_train, RSV_X_val, RSV_y_val, RSV_X_test, RSV_y_test = split_data(RSV_data, 'RSV')

In [10]:
daily_train, daily_val, daily_test = split_data(RSV_data, 'RSV')

In [11]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [12]:
RSV_X_train, RSV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
RSV_X_val, RSV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
RSV_X_test, RSV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [13]:
RSV_X_train.shape

(5025, 25, 29)

In [14]:
RSV_y_train.shape

(5025, 5)

In [15]:
# RSV_y_train = np.expand_dims(RSV_y_train, axis=-1)
# RSV_y_val = np.expand_dims(RSV_y_val, axis=-1)
# RSV_y_test = np.expand_dims(RSV_y_test, axis=-1)

In [16]:
# qloss function to calculate loss
def qloss(y_true, y_pred, n_q):
  q = np.array(range(1, n_q + 1))
  left = (q / (n_q + 1) - 1) * (y_true - y_pred)
  right = q / (n_q + 1) * (y_true - y_pred)

  return keras.backend.mean(keras.backend.maximum(left, right))

In [17]:
# Function to create the model
def get_model(input_dim, num_units, act, dp, gauss_std, num_hidden_layers, num_quantiles):
    input = Input((input_dim[1], input_dim[2], ), name='input')

    x = input

    for i in range(num_hidden_layers):
        x = Dense(num_units[i], use_bias=True, kernel_initializer='he_normal', bias_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.001), activation=act[i])(x)
        x = Dropout(dp[i])(x)
        x = GaussianNoise(gauss_std[i])(x)

    # Flatten the 3D input to 2D to remove the time dimension
    x = Flatten()(x)

    x = Dense(out_sam_win_sz, activation=None, use_bias=True, kernel_initializer='he_normal', bias_initializer='he_normal')(x)

    model = Model(input, x)

    return model

In [18]:
# Objective function for PSO
def objective_function(params):
    num_hidden_layers = int(params[0])
    # num_quantiles = int(params[1])
    num_quantiles = out_sam_win_sz

    # Adjust lists to match the number of hidden layers
    num_units = [int(params[1])] * num_hidden_layers
    act = ['relu'] * num_hidden_layers
    dropout = [params[2]] * num_hidden_layers
    gauss_std = [params[3]] * num_hidden_layers

    # Create the model
    model = get_model(input_dim=input_dim, num_units=num_units, act=act, dp=dropout, gauss_std=gauss_std, num_hidden_layers=num_hidden_layers, num_quantiles=num_quantiles)

    # Compile the model
    model.compile(loss=lambda y_t, y_p: qloss(y_t, y_p, n_q=num_quantiles), optimizer='adam')

    # Train the model
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    model.fit(RSV_X_train, RSV_y_train, epochs=80, validation_data=(RSV_X_val, RSV_y_val), batch_size=64, callbacks=[early_stopping], verbose=0)

    # Predictions
    predictions = model.predict(RSV_X_val)

    # Calculate mean squared error (MSE) as the objective function
    mse = mean_squared_error(RSV_y_val, predictions)

    return mse

In [19]:
# # Parameter bounds for PSO
# lb = [1, 10, 50, 50, 0.0, 0.0, 0.0, 0.0]  # Lower bounds [num_hidden_layers, num_quantiles, ...]
# ub = [5, 99, 300, 300, 0.5, 0.5, 1.0, 1.0]  # Upper bounds [num_hidden_layers, num_quantiles, ...]

# input_dim = 25

# swarmsize = 50
# maxiter = 100

In [20]:
# Parameter bounds for PSO
lb = [1, 30, 0.1, 0.0]  # Lower bounds [num_hidden_layers, num_quantiles, ...]
ub = [3, 80, 0.3, 0.5]  # Upper bounds [num_hidden_layers, num_quantiles, ...]

input_dim = RSV_X_train.shape

swarmsize = 20
maxiter = 50

In [21]:
start_time = time.time()

# Run PSO to find the best parameters
best_params, _ = pso(objective_function, lb, ub, swarmsize=swarmsize, maxiter=maxiter)

end_time = time.time()
execution_time = end_time - start_time

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
17/17 ━━━━━━━━━━━━

In [22]:
execution_time

2189.910897731781

In [23]:
best_params

array([ 3.        , 30.        ,  0.16745665,  0.        ])

In [24]:
# best_params = [1.49569993, 44.45113891, 0.11376749, 0]

In [25]:
# best_params = [2.44767625, 50.70353665, 0.26700986, 0.42920676]
# best_params = [1.71454653, 30.57286514, 0.21601151, 0.28231749]
# best_params = [1.05013429, 79.89863874,  0.13223594,  0.]

In [26]:
num_hidden_layers = int(best_params[0])
# num_quantiles = int(best_params[1])
num_quantiles = out_sam_win_sz

# Adjust lists to match the number of hidden layers
num_units = [int(best_params[1])] * num_hidden_layers
act = ['relu'] * num_hidden_layers
dropout = [best_params[2]] * num_hidden_layers
gauss_std = [best_params[3]] * num_hidden_layers

# Create the model
QRNN_model = get_model(input_dim=input_dim, num_units=num_units, act=act, dp=dropout, gauss_std=gauss_std, num_hidden_layers=num_hidden_layers, num_quantiles=num_quantiles)

# Compile the model
QRNN_model.compile(loss=lambda y_t, y_p: qloss(y_t, y_p, n_q=num_quantiles), optimizer='adam')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
QRNN_model.fit(RSV_X_train, RSV_y_train, epochs=80, validation_data=(RSV_X_val, RSV_y_val), batch_size=64, callbacks=[early_stopping], verbose=0)

In [27]:
# Predictions
RSV_test_predictions = QRNN_model.predict(RSV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [28]:
# # Train and predict with optimal parameters
# RSV_test_predictions = esn_model(optimal_params, RSV_X_train, RSV_y_train, RSV_X_test)

In [29]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

In [30]:
mse = mean_squared_error(RSV_y_test, RSV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(RSV_y_test, RSV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(RSV_y_test, RSV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 5.670471296659289e-06
RMSE: 0.002381275140898105
MAE: 0.0012753819099263614
MAPE: 6869.713290424379%


In [32]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/RSV/'

# df_pred = pd.DataFrame(RSV_test_predictions[:,0], columns=['qrnn_pred'])
# filename = 'QRNN_pred.csv'
# df_pred.to_csv(path_1 + filename)